In [1]:
import pandas as pd
import numpy as np
from pandas.io.json import json_normalize
import folium
from geopy.geocoders import Nominatim
import requests
import matplotlib.cm as cm
import matplotlib.colors as colors


In [2]:
address = 'Quan 7, Ho Chi Minh'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Manhattan are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Manhattan are 10.7365728, 106.7224322.


In [7]:
map_dn = folium.Map(location=[latitude, longitude], zoom_start=15)
map_dn

In [8]:
CLIENT_ID = 'ZRHSAYU5TZTMWAMXTJ3HTUEN5CHDJAI1ZXC3STWIPCUVIJZT' #Foursquare ID
CLIENT_SECRET = 'T2CXAM5OQ2UIINW1BUWDTOAW1W4IXOXB352VTCHJQKWFCA2P' #Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 1500 # define radius

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude, 
    longitude, 
    radius, 
    LIMIT)
url

results = requests.get(url).json()

In [9]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.shape

(58, 4)

In [10]:
for lat, lng, label in zip(nearby_venues['lat'], nearby_venues['lng'], nearby_venues['name']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_dn)  

from folium import plugins
hm_data = nearby_venues[["lat", "lng"]].to_numpy().tolist()
map_dn.add_child(plugins.HeatMap(hm_data))
map_dn

In [11]:
nearby_venues


,name,categories,lat,lng
0,Uraetei BBQ Japan 5 - Pho Dinh 浦江亭,BBQ Joint,10.730768,106.723235
1,Pizza 4P's,Pizza Place,10.731695,106.715756
2,Fit24 Fitness & Yoga Center - Thien Son Plaza,Gym / Fitness Center,10.732093,106.719297
3,Starbucks (Starbucks Crescent Residence 1),Coffee Shop,10.728165,106.718838
4,Hu tieu Nam Vang Ngoc Dung,Vietnamese Restaurant,10.737971,106.721484
5,Saigon Sports Club,Gym / Fitness Center,10.739537,106.728993
6,MerPerl Crystal Palace,Hotel,10.726712,106.723237
7,Kingscross,Bistro,10.727458,106.719559
8,Crescent Mall,Shopping Mall,10.728648,106.718497
9,Saigon South Golf,Golf Course,10.732649,106.715406
